In [1]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [2]:
DATASET_NAME = 'DBLP-Scholar'

In [3]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [4]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

[array([[ 0.02130077, -0.01306447,  0.16074979,  0.04089747, -0.02953706,
         0.16586197, -0.20704345, -0.05367793,  0.09883556,  0.20903153],
       [ 0.14984148, -0.16302353, -0.07959286,  0.02069081, -0.09310863,
        -0.12865009, -0.08660103,  0.12414484, -0.03437344,  0.1199733 ],
       [ 0.16826004, -0.00979924, -0.16610899, -0.11185468,  0.01434034,
        -0.00262906, -0.19765774, -0.1500956 ,  0.13790631,  0.04134799],
       [-0.01925926,  0.16185185,  0.03296296, -0.17555556,  0.1087037 ,
        -0.11351852,  0.03222222, -0.08055556,  0.11277778, -0.16259259],
       [-0.17319007, -0.02532617, -0.08007163,  0.03811717, -0.04783832,
         0.10079304,  0.22640061, -0.12023535,  0.01202354,  0.17600409]]), array([[ 0.02269101,  0.16114164, -0.08810495, -0.06257756, -0.00407729,
        -0.1145187 , -0.11398688,  0.10476866, -0.15865981,  0.1694735 ],
       [-0.00179937,  0.05892937,  0.00382366, -0.19860549,  0.07804768,
         0.06050382,  0.21322537, -0.14417

In [5]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


1
32


In [6]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [7]:
## Open dataset 
df = pd.read_csv('../../lsh-test-data/' + DATASET_NAME + '-embeddings.csv')

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(30):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 150)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

K: 1, L: 1, Precision:0.7345794392523365
K: 1, L: 2, Precision:0.974766355140187
K: 1, L: 3, Precision:0.994392523364486
K: 1, L: 4, Precision:0.9981308411214953
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:0.5317757009345795
K: 2, L: 2, Precision:0.9196261682242991
K: 2, L: 3, Precision:0.985981308411215
K: 2, L: 4, Precision:0.9962616822429906
K: 2, L: 5, Precision:0.9990654205607477
K: 2, L: 6, Precision:1.0
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.47757009345794393
K: 3, L: 2, Precision:0.8084112149532711
K: 3, L: 3, Precision:0.9345794392523364
K: 3, L: 4, Precision:0.9813084112149533
K: 3, L: 5, Precision:0.9906542056074766
K: 3, L: 6, Precision:0.9990654205607477
K: 3, L: 7, Precision:1.0
K: 3, L: 8, Precision:1.0
K: 3, L: 9, Precision:1.0
K: 3, L: 10, Precisio

K: 21, L: 4, Precision:0.1102803738317757
K: 21, L: 5, Precision:0.12429906542056075
K: 21, L: 6, Precision:0.12242990654205607
K: 21, L: 7, Precision:0.1542056074766355
K: 21, L: 8, Precision:0.17570093457943925
K: 21, L: 9, Precision:0.17757009345794392
K: 21, L: 10, Precision:0.197196261682243
K: 22, L: 1, Precision:0.014953271028037384
K: 22, L: 2, Precision:0.044859813084112146
K: 22, L: 3, Precision:0.07102803738317758
K: 22, L: 4, Precision:0.1102803738317757
K: 22, L: 5, Precision:0.10841121495327102
K: 22, L: 6, Precision:0.15233644859813084
K: 22, L: 7, Precision:0.11214953271028037
K: 22, L: 8, Precision:0.16355140186915887
K: 22, L: 9, Precision:0.17570093457943925
K: 22, L: 10, Precision:0.19906542056074766
K: 23, L: 1, Precision:0.018691588785046728
K: 23, L: 2, Precision:0.037383177570093455
K: 23, L: 3, Precision:0.06822429906542056
K: 23, L: 4, Precision:0.09158878504672897
K: 23, L: 5, Precision:0.08504672897196262
K: 23, L: 6, Precision:0.11588785046728972
K: 23, L: 